In [ ]:

#DataSet is available in below link
#http://stats.espncricinfo.com/ci/engine/records/team/series_results.html?id=159;type=trophy


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import time
import logging

In [2]:
def LaunchURL(URL):
    # create a new Firefox session
    driver1 = webdriver.Firefox(executable_path=r'C:\Users\shubh\Jupyter\BPL\geckodriver.exe')
    driver1.implicitly_wait(10)
    driver1.get(URL)
    return driver1

In [3]:
def SeriesPage(driver):
    return driver.find_elements_by_class_name('data-link')

In [4]:
def AllMatchDetails(driver):
    return driver.find_elements_by_class_name('potMatchLink')

In [5]:
def RedirectToCommetry(driver, Inning):
    NewURL = driver.current_url
    NewURL = NewURL.replace('/scorecard/','/commentary/')+'?innings=' +  str(Inning)
    driver.get(NewURL)
    return driver


In [6]:
def FindDivElement(driver):
    Frame = driver.find_element_by_id('pane-main')
    Frame2 = Frame.find_element_by_id('main-container')
    return Frame2.find_elements_by_class_name('main-content')[0]

In [7]:
def BatingTeamName(driver):
    return driver.find_elements_by_tag_name('button')[0].get_property('innerText')

In [8]:
def OppositionTeam(driver,Inning):
    
    if Inning ==1:
        return driver.find_elements_by_class_name('react-router-link')[13].get_property('innerText')
    else:
        return driver.find_elements_by_class_name('react-router-link')[12].get_property('innerText')

In [9]:
#launch url
logging.basicConfig(filename="newfile.log", 
                    format='%(asctime)s %(message)s', 
                    filemode='w')
logger=logging.getLogger() 
logger.setLevel(logging.DEBUG)
url = "http://stats.espncricinfo.com/ci/engine/records/team/series_results.html?id=159;type=trophy"
logger.info(url)
driverMain = LaunchURL(url)
logger.info("Main Thead Opened")

In [22]:
# get All Session data 
data = []
matchNumber =0
SeriesLinks = SeriesPage(driverMain)
logger.info("SeriesData Page Opened")
for EachSeries in SeriesLinks[8:]:
    #click on Each Series
    print ('-------------------------------------' + str(EachSeries.get_property('innerText')))
    logger.info(str(EachSeries.get_property('innerText') + " Series Started"))
    #EachSeries.click()
    SessionURL = EachSeries.get_attribute('href')
    driver = LaunchURL(SessionURL)
    logger.info ("New Thread Open")
    MatchLinks = AllMatchDetails(driver)
    for match in MatchLinks:
        # Click on Each Match
        #print (match.find_element_by_tag_name('a').get_attribute('href'))
        #match.click()
        #time.sleep(0.5)
        #matchURL = driver.current_url
        #try:
        matchNumber += 1
        driverTemp = LaunchURL(match.find_element_by_tag_name('a').get_attribute('href'))
        #driver.execute_script('window.history.go(-1)')
        # Re Direct to Commetry Tab
        Ground = driverTemp.find_elements_by_class_name('stadium-details')[0].get_property('innerText')
        for Inning in range(1,3):
        

            driverTemp = RedirectToCommetry(driverTemp, Inning)
            time.sleep(0.2)
            logger.info(str(match.find_element_by_tag_name('a').get_property('innerText'))+ " Opened in new thread")
            #Scroll Down
            #ScrollDown(driver)

            # Find Main Div Element
            div = FindDivElement(driverTemp)
            blnTaskComplete =False
            SCROLL_PAUSE_TIME = 0.5
            for i in range (0,15):
                html = driverTemp.find_element_by_tag_name('html')
                time.sleep(0.2)
                html.send_keys(Keys.END)
                time.sleep(0.2)
                html.send_keys(Keys.UP)
                time.sleep(0.2)
                html.send_keys(Keys.END)
            # get End Over Details
            # div.find_elements_by_tag_name('h4')
            # get Each Ball Details

            div = FindDivElement(driverTemp)
            for eachball in div.find_elements_by_class_name('commentary-item'):
                record = eachball.get_property('innerText')
                if ' to ' in record:
                    if str(str(record.splitlines()[0]).replace(" ","")[0]).isnumeric():
                        tempdata = str(matchNumber) + '\n' + Ground + '\n' + str(Inning) + '\n'+ BatingTeamName(div) + '\n'+ OppositionTeam(div,Inning) + '\n' + record
                        #print (tempdata)
                        data.append (tempdata.splitlines())
                        logger.info ("Data Extracted" + str(record))
                    else:
                        tempLine=''
                        print (record.split('\n') [2:])
                        for words in record.split('\n') [2:]:
                            if len(tempLine)==0:
                                tempLine = words
                            else:
                                tempLine += '\n' + words
                        print (tempLine)
                        tempdata = str(matchNumber) + '\n' + Ground + '\n' + str(Inning) + '\n'+ BatingTeamName(div) + '\n'+ OppositionTeam(div,Inning) + '\n' + tempLine
                        #print (tempdata.splitlines()[2:])
                        data.append (tempdata.splitlines())

                        logger.info ("Data Extracted from 2nd element" + str(record))
        driverTemp.close()
    driver.close()
    
        #except:
            #logger.info ("Error Occured and passed")
            #pass
logger.info ("Data Appened into DataFrame")
df = pd.DataFrame(data=data)
df.to_csv('BPL.csv')                

logger.info ("Data Saved")

-------------------------------------Bangladesh Premier League
['19.6\t', '6', 'Yasir Arafat to Styris, SIX runs,']
19.6	
6
Yasir Arafat to Styris, SIX runs,


In [41]:
div.find_elements_by_tag_name('button')[0].get_property('innerText')

'Royal Innings'

In [56]:
div.find_elements_by_class_name('react-router-link')[13].get_property('innerText')

'Burnr Innings'

In [76]:
a ="a,b,c,d,e,f"
a.split(",")[2:]

['c', 'd', 'e', 'f']